<a href="https://colab.research.google.com/github/GuillermoBastian/LatamProject/blob/main/solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# --------- Predicción de demora en vuelos - LATAM --------------

# 1. Importar Librerías

In [221]:
# Librería carga datos
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [255]:
#Librerías procesamiento datos y evaluación
import pandas as pd 
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [223]:
# Librerías gráficos
import seaborn as sns

In [224]:
#Librerías Modelamiento


# 2 Lectura & Añadir Campos Challenge

## 2.1 Load data

In [225]:
original_df = pd.read_csv('gdrive/My Drive/LATAM/dataset_SCL.csv', parse_dates=['Fecha-I', 'Fecha-O'])
feriados_chile = pd.read_excel("/content/gdrive/MyDrive/LATAM/Feriados_Chile.xlsx")
km_tpo = pd.read_excel("/content/gdrive/MyDrive/LATAM/KM_DISTANCIA.xlsx")
original_df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,1,1,2017,Domingo,I,American Airlines,Santiago,Miami
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,2,1,2017,Lunes,I,American Airlines,Santiago,Miami
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,3,1,2017,Martes,I,American Airlines,Santiago,Miami
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,4,1,2017,Miercoles,I,American Airlines,Santiago,Miami
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,5,1,2017,Jueves,I,American Airlines,Santiago,Miami


## 2.2 Borrar Variables que no agregan valor

In [226]:

cols_borrar = ["AÑO", "Ori-O", "Ori-I", "SIGLAORI"]

def delete_cols(df, cols_borrar):
  try:
    for k in cols_borrar:
      del df[k]
    print("OK: Columnas fueron borradas")
  except:
    print("ERROR: Columnas ya habían sido borradas")
    pass  

delete_cols(original_df, cols_borrar)

OK: Columnas fueron borradas


In [227]:
original_df.head(3)

,Fecha-I,Vlo-I,Des-I,Emp-I,Fecha-O,Vlo-O,Des-O,Emp-O,DIA,MES,DIANOM,TIPOVUELO,OPERA,SIGLADES
0,2017-01-01 23:30:00,226,KMIA,AAL,2017-01-01 23:33:00,226,KMIA,AAL,1,1,Domingo,I,American Airlines,Miami
1,2017-01-02 23:30:00,226,KMIA,AAL,2017-01-02 23:39:00,226,KMIA,AAL,2,1,Lunes,I,American Airlines,Miami
2,2017-01-03 23:30:00,226,KMIA,AAL,2017-01-03 23:39:00,226,KMIA,AAL,3,1,Martes,I,American Airlines,Miami


## 2.4 Depurar missings!

In [228]:
original_df.isna().sum() > 0

Fecha-I      False
Vlo-I        False
Des-I        False
Emp-I        False
Fecha-O      False
Vlo-O         True
Des-O        False
Emp-O        False
DIA          False
MES          False
DIANOM       False
TIPOVUELO    False
OPERA        False
SIGLADES     False
dtype: bool

In [229]:
#Solo faltaba un dato del Nro. de vuelo, lo completaremos con el mismo código de Santiago a Lima con LAW para día jueves
original_df["Vlo-O"] = original_df["Vlo-O"].fillna(704)

## 2.3 Agregar Variables challenge

In [230]:
#1.- Calcular temporada_alta  -- si Fecha-I está entre 15-Dic y 3-Mar, o 15-Jul y 31-Jul, o 11-Sep y 30-Sep, 0 si no.
original_df["temporada_alta"] = 0
original_df.loc[(original_df['Fecha-I'].dt.strftime('%m-%d') >= '12-15') | (original_df['Fecha-I'].dt.strftime('%m-%d') <= '03-03'), "temporada_alta"] = 1
original_df.loc[(original_df['Fecha-I'].dt.strftime('%m-%d') >= '07-15') & (original_df['Fecha-I'].dt.strftime('%m-%d') <= '07-31'), "temporada_alta"] = 1
original_df.loc[(original_df['Fecha-I'].dt.strftime('%m-%d') >= '09-11') & (original_df['Fecha-I'].dt.strftime('%m-%d') <= '09-30'), "temporada_alta"] = 1

#2.- Calcular dif_min 
original_df["dif_min"] = (original_df["Fecha-O"] - original_df["Fecha-I"])
original_df["dif_min"]  = original_df["dif_min"]/np.timedelta64(1,'m')

# 3.- Calcular atraso_15 
original_df["atraso_15"] = 0
original_df.loc[original_df["dif_min"] > 15, "atraso_15"] = 1

#4.- Calcular periodo_dia : mañana : 0 | tarde : 1 | noche : 2
original_df["periodo_dia"] = 0
original_df.loc[(original_df['Fecha-I'].dt.strftime('%H:%M') >= '05:00') & (original_df['Fecha-I'].dt.strftime('%H:%M') <= '11:59'), "periodo_dia"] = 0
original_df.loc[(original_df['Fecha-I'].dt.strftime('%H:%M') >= '12:00') & (original_df['Fecha-I'].dt.strftime('%H:%M') <= '18:59'), "periodo_dia"] = 1
original_df.loc[(original_df['Fecha-I'].dt.strftime('%H:%M') >= '19:00') | (original_df['Fecha-I'].dt.strftime('%H:%M') <= '04:59'), "periodo_dia"] = 2


original_df.to_csv("synthetic_features.csv")

In [231]:
original_df.head(3)

,Fecha-I,Vlo-I,Des-I,Emp-I,Fecha-O,Vlo-O,Des-O,Emp-O,DIA,MES,DIANOM,TIPOVUELO,OPERA,SIGLADES,temporada_alta,dif_min,atraso_15,periodo_dia
0,2017-01-01 23:30:00,226,KMIA,AAL,2017-01-01 23:33:00,226,KMIA,AAL,1,1,Domingo,I,American Airlines,Miami,1,3.0,0,2
1,2017-01-02 23:30:00,226,KMIA,AAL,2017-01-02 23:39:00,226,KMIA,AAL,2,1,Lunes,I,American Airlines,Miami,1,9.0,0,2
2,2017-01-03 23:30:00,226,KMIA,AAL,2017-01-03 23:39:00,226,KMIA,AAL,3,1,Martes,I,American Airlines,Miami,1,9.0,0,2


## 2.4 Añadir Variables propias

In [232]:
feriados_chile.head(3)

,Evento,Mes,Dia
0,ano_nuevo,1.0,1.0
1,viernes_santo,4.0,14.0
2,sabado_santo,4.0,15.0


In [233]:
# 1.- Feriados:
main_nfields_df = pd.merge(original_df, feriados_chile,  how='left', left_on=['DIA','MES'], right_on = ['Dia','Mes'])
cols_borrar = ['Dia','Mes']
delete_cols(main_nfields_df, cols_borrar)
main_nfields_df['Evento'] = main_nfields_df['Evento'].fillna(0)

# 2.- Hora de partida
main_nfields_df["HORA"] = main_nfields_df["Fecha-I"].dt.hour + main_nfields_df["Fecha-I"].dt.minute/60

# 3.- Emp - Vuelo
main_nfields_df["Emp-Vlo"] = main_nfields_df["Emp-I"].astype(str) + main_nfields_df["Vlo-I"].astype(str)
main_nfields_df.head()

# 3.- Km diferencia + 4.- Tiempo de vuelo
main_nfields_df = pd.merge(main_nfields_df, km_tpo,  how='left', left_on=['SIGLADES'], right_on = ['SIGLADES'])
cols_borrar = ['Duración_hrs']
delete_cols(main_nfields_df, cols_borrar)
main_nfields_df

# 6.- último dia del mes

def calc_ult_dia (anio):
  lista_ult_dia = []
  lista_mes = []
  for i in range(1,13): 
    dummy_date = "{}-01-{}".format(str(i),str(anio))
    lista_mes.append(i)
    lista_ult_dia.append(pd.Period(dummy_date,freq='M').end_time.date().day)
    df_ult_dia = pd.DataFrame(list(zip(lista_mes, lista_ult_dia)),
               columns =['MES', 'ULT_DIA'])
  return df_ult_dia 

df_ult_dia = calc_ult_dia(2017)

main_nfields_df = pd.merge(main_nfields_df, df_ult_dia,  how='left', left_on=['MES'], right_on = ['MES'])
main_nfields_df.head()


OK: Columnas fueron borradas
OK: Columnas fueron borradas


,Fecha-I,Vlo-I,Des-I,Emp-I,Fecha-O,Vlo-O,Des-O,Emp-O,DIA,MES,...,temporada_alta,dif_min,atraso_15,periodo_dia,Evento,HORA,Emp-Vlo,KM,DURACION_MIN,ULT_DIA
0,2017-01-01 23:30:00,226,KMIA,AAL,2017-01-01 23:33:00,226,KMIA,AAL,1,1,...,1,3.0,0,2,ano_nuevo,23.5,AAL226,6627,497.0,31
1,2017-01-02 23:30:00,226,KMIA,AAL,2017-01-02 23:39:00,226,KMIA,AAL,2,1,...,1,9.0,0,2,0,23.5,AAL226,6627,497.0,31
2,2017-01-03 23:30:00,226,KMIA,AAL,2017-01-03 23:39:00,226,KMIA,AAL,3,1,...,1,9.0,0,2,0,23.5,AAL226,6627,497.0,31
3,2017-01-04 23:30:00,226,KMIA,AAL,2017-01-04 23:33:00,226,KMIA,AAL,4,1,...,1,3.0,0,2,0,23.5,AAL226,6627,497.0,31
4,2017-01-05 23:30:00,226,KMIA,AAL,2017-01-05 23:28:00,226,KMIA,AAL,5,1,...,1,-2.0,0,2,0,23.5,AAL226,6627,497.0,31


In [234]:
#Borrar últiomas columnas 
cols_borrar = ["Fecha-I", "Fecha-O"]
delete_cols(main_nfields_df, cols_borrar)

OK: Columnas fueron borradas


# 3 Análisis de datos

In [235]:
#Reconocer si la columna es categorica o numérica
def reconocer_cate_num(df, total_cols):
  numerical_columns = []
  categorical_columns = []

  for i in total_cols:
    if df[i].dtype == object:
      categorical_columns.append(i)
    else:
      numerical_columns.append(i)
  return categorical_columns, numerical_columns

total_cols = main_nfields_df.columns
categorical_columns, numerical_columns = reconocer_cate_num(main_nfields_df, total_cols)

main_nfields_df[categorical_columns] = main_nfields_df[categorical_columns].astype(str)

In [236]:
analysis_df = main_nfields_df

In [237]:
# Label Encoder
# from sklearn.preprocessing import OrdinalEncoder
# oe = OrdinalEncoder()
# categorical_df = oe.fit_transform(analysis_df[categorical_columns])
# analysis_df[categorical_columns] = categorical_df

In [238]:
# Reconocer variables independientes del target
ycols = ["atraso_15"]

def get_complement_columns(total_cols, excluded_cols):
  xcols= []
  for i in total_cols:
    if i not in ycols:
      xcols.append(i)
  return xcols

xcols = get_complement_columns(total_cols, ycols)

In [239]:
analysis_df.describe()

,DIA,MES,temporada_alta,dif_min,atraso_15,periodo_dia,HORA,KM,DURACION_MIN,ULT_DIA
count,68206.000000,68206.000000,68206.000000,68206.000000,68206.000000,68206.000000,68206.000000,68206.000000,68206.000000,68206.000000
mean,15.714790,6.622585,0.333724,9.110855,0.184940,0.882650,12.899395,2179.883940,181.264757,30.437381
std,8.782886,3.523321,0.471546,19.313387,0.388252,0.782485,5.805976,2341.414645,163.920700,0.857487
min,1.000000,1.000000,0.000000,-14.000000,0.000000,0.000000,0.000000,301.000000,43.000000,28.000000
25%,8.000000,3.000000,0.000000,-1.000000,0.000000,0.000000,8.000000,916.000000,94.000000,30.000000
50%,16.000000,7.000000,0.000000,4.000000,0.000000,1.000000,13.000000,1221.000000,116.000000,31.000000
75%,23.000000,10.000000,1.000000,12.000000,0.000000,2.000000,17.766667,2453.000000,203.000000,31.000000
max,31.000000,12.000000,1.000000,161.000000,1.000000,2.000000,23.983333,11903.000000,894.000000,31.000000


In [240]:
analysis_df[xcols].corr()["dif_min"]

#Resultados - VARIABLES:
# MES
# TIPOVUELO
# Vlo-I / Vlo-O
# SIGLADES

DIA              -0.006706
MES               0.077280
temporada_alta    0.020899
dif_min           1.000000
periodo_dia       0.035625
HORA              0.055772
KM               -0.012589
DURACION_MIN     -0.006666
ULT_DIA           0.042370
Name: dif_min, dtype: float64

In [195]:
analysis_df

,Fecha-I,Vlo-I,Des-I,Emp-I,Fecha-O,Vlo-O,Des-O,Emp-O,DIA,MES,...,temporada_alta,dif_min,atraso_15,periodo_dia,Evento,HORA,Emp-Vlo,KM,DURACION_MIN,ULT_DIA
0,2017-01-01 23:30:00,226,KMIA,AAL,2017-01-01 23:33:00,226,KMIA,AAL,1,1,...,1,3.0,0,2,ano_nuevo,23.500000,AAL226,6627,497.0,31
1,2017-01-02 23:30:00,226,KMIA,AAL,2017-01-02 23:39:00,226,KMIA,AAL,2,1,...,1,9.0,0,2,0,23.500000,AAL226,6627,497.0,31
2,2017-01-03 23:30:00,226,KMIA,AAL,2017-01-03 23:39:00,226,KMIA,AAL,3,1,...,1,9.0,0,2,0,23.500000,AAL226,6627,497.0,31
3,2017-01-04 23:30:00,226,KMIA,AAL,2017-01-04 23:33:00,226,KMIA,AAL,4,1,...,1,3.0,0,2,0,23.500000,AAL226,6627,497.0,31
4,2017-01-05 23:30:00,226,KMIA,AAL,2017-01-05 23:28:00,226,KMIA,AAL,5,1,...,1,-2.0,0,2,0,23.500000,AAL226,6627,497.0,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68201,2017-12-22 14:55:00,400,SPJC,JAT,2017-12-22 15:41:00,400.0,SPJC,JAT,22,12,...,1,46.0,1,1,0,14.916667,JAT400,2453,203.0,31
68202,2017-12-25 14:55:00,400,SPJC,JAT,2017-12-25 15:11:00,400.0,SPJC,JAT,25,12,...,1,16.0,1,1,navidad,14.916667,JAT400,2453,203.0,31
68203,2017-12-27 14:55:00,400,SPJC,JAT,2017-12-27 15:35:00,400.0,SPJC,JAT,27,12,...,1,40.0,1,1,0,14.916667,JAT400,2453,203.0,31
68204,2017-12-29 14:55:00,400,SPJC,JAT,2017-12-29 15:08:00,400.0,SPJC,JAT,29,12,...,1,13.0,0,1,0,14.916667,JAT400,2453,203.0,31


In [196]:
flights = main_nfields_df.pivot("MES", "Vlo-I" , "dif_min")
ax = sns.heatmap(flights)

ValueError: ignored

# 4. Pre-Procesamiento:  
4.1 One Hot - Variables Categoricas <br>
4.2 Normalizacion - Variables Numéricas <br>

## 4.1 One - Hot: Categoricas

In [241]:
main_nfields_df["Emp-I"]

0        AAL
1        AAL
2        AAL
3        AAL
4        AAL
        ... 
68201    JAT
68202    JAT
68203    JAT
68204    JAT
68205    JAT
Name: Emp-I, Length: 68206, dtype: object

In [242]:
one_hot_status= 'N'

In [243]:

if one_hot_status == 'N':
  for i in categorical_columns:
    main_onehot_df = pd.get_dummies(main_nfields_df[i], prefix=i)
    one_hot_status= 'Y'


main_onehot_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68206 entries, 0 to 68205
Columns: 685 entries, Emp-Vlo_AAL226 to Emp-Vlo_UAL846A
dtypes: uint8(685)
memory usage: 45.1 MB


# 4.2 Normalizar variables numéricas

In [244]:
def normalizar_df(df):
  scaler = StandardScaler()
  scaler.fit(df)
  scaled = scaler.fit_transform(df)
  scaled_df = pd.DataFrame(scaled, columns=df.columns)
  return scaled_df

In [246]:
cols = ["dif_min", "HORA"]
norm_df = normalizar_df(main_nfields_df[numerical_columns])
norm_df.shape

(68206, 10)

# 5. Train - Test Split

In [252]:
input_model = main_nfields_df
input_model_onehot = pd.concat([main_nfields_df[numerical_columns] , main_onehot_df], axis=1)
input_model_norm = pd.concat([main_nfields_df[categorical_columns], norm_df] , axis=1)
input_model_onehot_norm = pd.concat([main_onehot_df, norm_df] , axis=1)



In [273]:
# Reconocer variables independientes del target
ycols = ["atraso_15", "dif_min"]
y_cols_clasiff = ["atraso_15"]

xcols_input = get_complement_columns(input_model.columns, ycols)
xcols_onehot = get_complement_columns(input_model_onehot.columns, ycols)
xcols_norm = get_complement_columns(input_model_norm.columns, ycols)
xcols_onehot_norm = get_complement_columns(input_model_onehot_norm.columns, ycols)


In [286]:
ts = 0.25

X_train, X_test, y_train, y_test = train_test_split(input_model[xcols_input], input_model[y_cols_clasiff], test_size=ts, random_state=42, stratify = input_model[y_cols_clasiff])
X_train_oh, X_test_oh, y_train_oh, y_test_oh = train_test_split(input_model_onehot[xcols_onehot], input_model_onehot[y_cols_clasiff], test_size=ts, random_state=42, stratify = input_model_onehot[y_cols_clasiff])
X_train_nor, X_test_nor, y_train_nor, y_test_nor = train_test_split(input_model_norm[xcols_norm], input_model_norm[y_cols_clasiff], test_size=ts, random_state=42, stratify = input_model_norm[y_cols_clasiff])
X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(input_model_onehot_norm[xcols_onehot_norm], input_model_onehot_norm[y_cols_clasiff], test_size=ts, random_state=42, stratify = input_model_onehot_norm[y_cols_clasiff])


In [282]:
y_train.value_counts(normalize = True)*100

atraso_15
0            81.506823
1            18.493177
dtype: float64